#### Import library yang dibutuhkan

In [227]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score
from imblearn.over_sampling import SMOTE

#### Variable yang dibutuhkan

In [235]:
dataset_path = "dataset/lapor.csv"
max_features=10
pca_num_components=5
data_path="data/"
number_of_neighbors=1000
number_of_fold=10

#### Preparasi data

In [245]:
train_df = pd.read_csv(dataset_path, delimiter=',')
train_df.head()

,id,TrackingID,Pelapor,JudulLaporan,IsiLaporan,Kategori,KategoriID,DisposisiInstansi,DisposisiInstansiID,Area,...,TanggalDisposisi,TanggalLaporanMasuk,TanggalLaporanAktivitasTerakhir,TanggalLaporanDitutup,TimestampLaporanMasuk,TimestampTanggalDisposisi,TimestampTanggalAktivitasTerakhir,TimestampTanggalLaporanDitutup,created_at,updated_at
0,1,22022,628214771xxxx,Tenaga Dokter di Nusa Tenggara Timur,Pak Menteri Kesehatan. Tolong perhatikan kami ...,Bidang Kesejahteraan Rakyat,16,Kementerian Kesehatan,118,Nusa Tenggara Timur,...,04 June 2012 11:24:06,28 May 2012 21:47:41,04 June 2012 11:24:06,27 July 2012 01:00:02,1338216461,1338783846,1338783846,1.343326e+09,2015-12-04 02:08:00,2015-12-04 02:08:00
1,2,22033,628574613xxxx,Standarisasi Pornografi & Pornoaksi,"Kemenkumham, standarisasi minimal pornografi &...",Teknologi Informasi dan Komunikasi,13,Kementerian Hukum dan HAM,248,Nasional,...,13 July 2012 18:13:34,29 May 2012 10:14:24,03 September 2014 12:45:38,25 September 2012 01:00:01,1338261264,1342178014,1409723138,1.348510e+09,2015-12-04 02:08:00,2015-12-04 02:08:00
2,3,22039,628214771xxxx,Biaya Operasi untuk Pasien Tidak Mampu,"Menkes RI yang terhormat, tolong inspeksi ke R...",Kesehatan,5,Kementerian Kesehatan,118,Nusa Tenggara Timur,...,04 June 2012 14:31:27,29 May 2012 11:41:25,22 January 2015 17:42:25,27 July 2012 01:00:02,1338266485,1338795087,1421923345,1.343326e+09,2015-12-04 02:08:00,2015-12-04 02:08:00
3,4,22067,628139296xxxx,Pembayaran Gaji Dokter PTT di Maluku Utara,"Selamat pagi, saya Andrianto, dokter PTT di pe...",Kesehatan,5,Kementerian Kesehatan,118,Maluku Utara,...,30 May 2012 19:11:38,30 May 2012 09:30:47,04 January 2015 04:13:23,08 January 2013 01:00:02,1338345047,1338379898,1420319603,1.357582e+09,2015-12-04 02:08:00,2015-12-04 02:08:00
4,5,22145,628215460xxxx,Pembalakan Liar di Kalimantan Tengah,Yth. Bpk. Kapolda Kalimantan Tengah. Penegak h...,Lingkungan Hidup dan Penanggulangan Bencana,11,Badan Reserse Kriminal,865,Kalimantan Tengah,...,15 June 2012 16:20:36,31 May 2012 16:40:41,06 September 2015 06:59:50,25 June 2012 12:53:12,1338457241,1339752036,1441497590,1.340604e+09,2015-12-04 02:08:00,2015-12-04 02:08:00


In [247]:
kategori_total_list = train_df["Kategori"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(kategori_total_list)

Bantuan Langsung Sementara Masyarakat (BLSM)                                 14739
Topik Lainnya                                                                 9175
Infrastruktur                                                                 6949
Reformasi Birokrasi dan Tata Kelola                                           6840
Bantuan Siswa Miskin (BSM)                                                    6608
Beras Miskin (Raskin)                                                         5952
Kepesertaan Non-KPS                                                           4557
Pendidikan                                                                    3506
Lingkungan Hidup dan Penanggulangan Bencana                                   2748
Kartu Indonesia Pintar (KIP)                                                  2391
Kesehatan                                                                     2220
Kepesertaan KPS                                                               2036
Impl

#### Klasifikasi ulang berdasarkan kemiripan

In [137]:
kategori = train_df["Kategori"].values
kementerian = train_df["Kategori"].values
for i in range(len(kategori)):
    kat = str(kategori[i]).lower()
    if "kepegawaian" in kat:
        kementerian[i] = "Badan Kepegawaian Negara"
    elif "bencana" in kat:
        kementerian[i] = "Badan Nasional Penanggulangan Bencana"
    elif "korupsi" in kat:
        kementerian[i] = "KPK"
    elif "kepolisian" in kat:
        kementerian[i] = "Kepolisian"
    elif "dikti" in kat:
        kementerian[i] = "Menteri Riset dan Pendidikan Tinggi"
    elif "siswa" in kat or "pendidikan" in kat or "pintar" in kat or "kebudayaan" in kat:
        kementerian[i] = "Menteri Pendidikan dan Kebudayaan"
    elif "birokrasi" in kat or "pelayanan masyarakat" in kat:
        kementerian[i] = "Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi"
    elif "infrastruktur" in kat or "permukiman" in kat or "air minum" in kat:
        kementerian[i] = "Menteri Pekerjaan Umum dan Perumahan Rakyat"
    elif "raskin" in kat or "kps" in kat or "pkh" in kat or "kemiskinan" in kat or "sejahtera" in kat:
        kementerian[i] = "Menteri Sosial"
    elif "kehutanan" in kat or "lingkungan" in kat:
        kementerian[i] = "Menteri Lingkungan Hidup dan Kehutanan"
    elif "obat" in kat or "kesehatan" in kat or "sehat" in kat or "jamkesmas" in kat:
        kementerian[i] = "Menteri Kesehatan"
    elif "pertambangan" in kat or "bbm" in kat or "energi" in kat or "listrik" in kat or "migas" in kat:
        kementerian[i] = "Menteri ESDM"
    elif "perhubungan" in kat:
        kementerian[i] = "Menteri Perhubungan"
    elif "administrasi kependudukan" in kat or "pelayanan administrasi" in kat:
        kementerian[i] = "Menteri Dalam Negeri"    
    elif "teknologi informasi" in kat:
        kementerian[i] = "Menteri Telekomunikasi dan Informatika"
    elif "imigrasi" in kat:
        kementerian[i] = "Menteri Luar Negeri"
    elif "ketenagakerjaan" in kat:
        kementerian[i] = "Menteri Ketenagakerjaan"
    elif "pertanian" in kat:
        kementerian[i] = "Menteri Pertanian"
    elif "pos" in kat:
        kementerian[i] = "Menteri BUMN"
    elif "pajak" in kat or "bea" in kat:
        kementerian[i] = "Menteri Keuangan"
    elif "pendaftaran badan usaha" in kat or "pemasyarakatan" in kat:
        kementerian[i] = "Menteri Hukum dan Ham"
    elif "desa" in kat or "daerah tertinggal" in kat or "tertinggal" in kat or "transmigrasi" in kat:
        kementerian[i] = "Menteri Desa, Pembangunan Daerah Tertinggal dan Transmigrasi" 
    elif "koperasi" in kat or "ukm" in kat:
        kementerian[i] = "Menteri Koperasi dan Usaha Kecil dan Menengah"
    elif "kemaritiman" in kat:
        kementerian[i] = "Menteri Kelautan dan Perikanan"
    elif "haji" in kat:
        kementerian[i] = "Menteri Agama"
    elif "kesejahteraan" in kat:
        kementerian[i] = "Menko Pembangunan Manusia dan Kebudayaan"
    elif "kewaspadaan" in kat or "politik" in kat or "hukum" in kat or "keamanan" in kat or "polhukam" in kat:
        kementerian[i] = "Menko Polhukam"
    elif "ekonomi" in kat or "perekonomian" in kat or "perdagangan" in kat or "perindustrian" in kat or "investasi" in kat:
        kementerian[i] = "Menko Perekonomian"
    else:
        kementerian[i] = "Lainnya"

In [138]:
train_df["Kementerian"] = kementerian

In [139]:
kementerian_total_list = train_df["Kementerian"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (kementerian_total_list)

Lainnya                                                          28139
Menteri Sosial                                                   14172
Menteri Pendidikan dan Kebudayaan                                12846
Menteri Pekerjaan Umum dan Perumahan Rakyat                       7768
Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi     7164
Menteri Kesehatan                                                 3615
Menteri ESDM                                                      3264
Badan Nasional Penanggulangan Bencana                             2796
Menteri Dalam Negeri                                              1453
Menko Polhukam                                                    1208
Menteri Perhubungan                                               1134
Menko Perekonomian                                                 762
Menteri Lingkungan Hidup dan Kehutanan                             442
Menteri Telekomunikasi dan Informatika                             428
Menter

In [140]:
laporan = train_df["IsiLaporan"].values.astype('U')
laporan

array(['Pak Menteri Kesehatan. Tolong perhatikan kami RSU di daerah NTT, tenaga Dokter kurang & pelayanan tidak maksimal, tapi bisa melayani 3 s/d 4 tempat. Sangat berbahaya... Trims.',
       'Kemenkumham, standarisasi minimal pornografi & pornoaksi tolong dibuat dan disosialisasikan kepada masyarakat.',
       'Menkes RI yang terhormat, tolong inspeksi ke RSUD WZ. Yohanis, Kupang. Jika pasien tidak mampu, operasi penyakit dalam diminta bayar sekitar 10 juta, meskipun ada keterangan tidak mampu. Apakah ini benar?',
       ...,
       'Lstrik di ribuan pelanggan PLN Desa Brabe Kec. Maron Kab. Probolinggo, Jatim, Wilayah Kerja PLN Kraksaan Kab. Prob Jatim sering padam berjam-jam, antara lain tgl 15,24,25 September, 5,13 Oktober,9,10,11,13,16,30 November,1 Desember padam dan saat ini 3 De',
       'Kepada yth pengurus BPJSTK, saya mohon bantuannya, waktu saya daftar pertama kali menggunakan nmr HP 089601729737, namun sekarang no tsb b sudah tidak aktip, ketika saya ingin cek saldo bpjs d

In [141]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
for index, data in tqdm(enumerate(laporan)):
    clean=re.sub('[^ a-zA-Z]','',data.lower())
    stop=stopword.remove(clean)
    laporan[index] = stop
laporan

array(['pak menteri kesehatan perhatikan rsu daerah ntt tenaga dokter kurang  pelayanan maksimal bisa melayani  sd  tempat sangat berbahaya trims',
       'kemenkumham standarisasi minimal pornografi  pornoaksi dibuat disosialisasikan masyarakat',
       'menkes ri terhormat inspeksi rsud wz yohanis kupang pasien mampu operasi penyakit diminta bayar  juta meskipun keterangan mampu ini benar',
       ...,
       'lstrik ribuan pelanggan pln desa brabe kec maron kab probolinggo jatim wilayah kerja pln kraksaan kab prob jatim sering padam berjamjam lain tgl  september  oktober november desember padam saat  de',
       'yth pengurus bpjstk mohon bantuannya waktu daftar pertama kali menggunakan nmr hp  sekarang no tsb b tidak aktip saya cek saldo bpjs hp jadi kesulitan tidak menggunakan no hp yg baru',
       'hari jumat tanggal  oktober  mendaftarkan pemecahan pemisahan sertipikat tanah kantor bpn jakarta barat nomer berkas  terakhir cek kantor bpn jakarta barat hari jumat tanggal  novembe

In [ ]:
vectorizer = TfidfVectorizer(max_features=max_features)
X = vectorizer.fit_transform(laporan)
X.shape

In [143]:
feature=X.todense()
ftr = PCA(n_components=pca_num_components).fit_transform(feature)
ftr

array([[-0.02295902, -0.04967939,  0.08032994,  0.03676032,  0.09438705],
       [ 0.00032627, -0.02528172,  0.04505265, -0.00770379,  0.07944125],
       [ 0.02876365, -0.06836948,  0.0234573 , -0.01597515,  0.06618244],
       ...,
       [ 0.06404366,  0.02920866,  0.0059062 , -0.17093522,  0.03819583],
       [-0.04285838, -0.05113869,  0.11248707,  0.01455822,  0.06250055],
       [-0.12950323, -0.0004442 , -0.0264122 ,  0.02584432,  0.03786878]])

In [200]:
def strToOneHot(kementerian):
    one_hot_cls = []
    kementerian_u = pd.unique(kementerian)
    kementerian_u = kementerian_u.tolist()
    for i in range(len(kementerian)):
        one_hot = np.zeros((len(kementerian_u),), dtype=int)
        idx = kementerian_u.index(kementerian[i])
        one_hot[idx] = 1
        one_hot_cls.append(one_hot)
    one_hot_cls = np.array(one_hot_cls)
    return one_hot_cls

def oneHotToStr(one_hot_cls):
    cls=[]
    kementerian_u = pd.unique(kementerian)
    kementerian_u = kementerian_u.tolist()
    for data in one_hot_cls:
        index=np.argmax(data)
        cls.append(kementerian_u[index])
    return cls

In [145]:
np.save(data_path+'feature.npy',ftr)
np.save(data_path+'kelas.npy',kementerian)

In [201]:
ftr=np.load(data_path+'feature.npy')
kementerian=np.load(data_path+'kelas.npy')

In [202]:
one_hot_cls=strToOneHot(kementerian)

In [203]:
print(ftr.shape)
print(one_hot_cls.shape)

(87766, 5)
(87766, 28)


In [204]:
indexed_cls = list(map(lambda x: np.argmax(x), one_hot_cls))
print(pd.unique(indexed_cls))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]


In [205]:
def valuesCount(arr):
    cls_dict = {}
    for i in arr:
        if i in cls_dict:
            cls_dict[i] += 1
        else:
            cls_dict[i] = 1
    return cls_dict
valuesCount(indexed_cls)

{0: 14172,
 1: 428,
 2: 3615,
 3: 2796,
 4: 12846,
 5: 7164,
 6: 762,
 7: 218,
 8: 1208,
 9: 3264,
 10: 28139,
 11: 40,
 12: 7768,
 13: 120,
 14: 1134,
 15: 350,
 16: 442,
 17: 8,
 18: 363,
 19: 1453,
 20: 27,
 21: 218,
 22: 340,
 23: 81,
 24: 195,
 25: 151,
 26: 399,
 27: 65}

In [206]:
ftr_used=ftr
cls_used=oneHotToStr(one_hot_cls)

In [207]:
X_train, X_test, y_train, y_test = train_test_split(ftr_used,cls_used, test_size=0.3, random_state=0, stratify=cls_used)

### SMOTE

In [208]:
indexed_cls = list(map(lambda x: np.argmax(x), strToOneHot(y_train)))
print(pd.unique(indexed_cls))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]


In [209]:
def valuesCount(arr):
    cls_dict = {}
    for i in arr:
        if i in cls_dict:
            cls_dict[i] += 1
        else:
            cls_dict[i] = 1
    return cls_dict
valuesCount(indexed_cls)

{0: 5015,
 1: 9920,
 2: 846,
 3: 5438,
 4: 19697,
 5: 8992,
 6: 1017,
 7: 238,
 8: 2285,
 9: 1957,
 10: 2530,
 11: 794,
 12: 279,
 13: 136,
 14: 153,
 15: 309,
 16: 533,
 17: 245,
 18: 300,
 19: 45,
 20: 153,
 21: 57,
 22: 254,
 23: 106,
 24: 84,
 25: 19,
 26: 6,
 27: 28}

In [215]:
sm = SMOTE(random_state=42)
ftr_res, cls_res = sm.fit_sample(X_train, y_train)

In [217]:
indexed_cls = list(map(lambda x: np.argmax(x), strToOneHot(cls_res)))
print(pd.unique(indexed_cls))
print(valuesCount(indexed_cls))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
{0: 19697, 1: 19697, 2: 19697, 3: 19697, 4: 19697, 5: 19697, 6: 19697, 7: 19697, 8: 19697, 9: 19697, 10: 19697, 11: 19697, 12: 19697, 13: 19697, 14: 19697, 15: 19697, 16: 19697, 17: 19697, 18: 19697, 19: 19697, 20: 19697, 21: 19697, 22: 19697, 23: 19697, 24: 19697, 25: 19697, 26: 19697, 27: 19697}


In [243]:
KNN=KNeighborsClassifier(n_neighbors=number_of_neighbors)
KNN=KNN.fit(ftr_res,cls_res)

In [234]:
clf=svm.SVC()
clf=clf.fit(ftr_res,indexed_cls)

In [244]:
y_predict=KNN.predict(X_test)

In [240]:
# X_test_predict = list(map(lambda x: np.argmax(x), strToOneHot(X_test)))
svm_predict=clf.predict(X_test)

In [241]:
y_test_indexed = list(map(lambda x: np.argmax(x), strToOneHot(y_test)))
confusion_matrix(y_test_indexed,svm_predict)

array([[   7, 2311,   16,    7,  911,  374,   15,    4,   20,   42,   32,
           6,   16,   19,    1,    4,    0,    0,    3,   41,   63,   54,
          18,   63,   59,   36,   13,  117],
       [  45,    1,  193,  330,    2,    2,    1,  104,   22,  510,    7,
         221,   38,   35,    2,  117,    0,    0,    9,  214,   34,    8,
          17,  112,   43,   77,   16,  170],
       [   3,    1,   18,    8,    1,    0,    0,   55,  644,   28,    3,
          10,   10,   20,    0,    8,    0,    0,    5,    3,   36,    3,
           2,   11,   36,   39,    5,   30],
       [  68, 1129,  114,  159, 3030,  439,   92,   64,   39,  878,   94,
         256,  150,   81,    3,  142,    0,    0,    9,  530,   27,   70,
         100,  327,   52,   62,   53,  474],
       [  21,  239,    7,    1,   72, 2408,   33,    9,    2,   24,  214,
           5,   85,   38,   10,    6,    0,    0,    5,  142,   23,  153,
          44,   72,    6,   74,   69,   92],
       [  55,    1,  108,   75,    

In [242]:
accuracy_score(y_test,y_predict)

0.36973034561336876